In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0039520263671875
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 10.171839246364232
200000 20.31935065298204
300000 18.24434361811352
400000 17.38540536962153
500000 19.404037095688103
600000 18.194381844577055
700000 17.967977011454227
800000 17.825746744747576
900000 9.564303799058665
1000000 14.148576434137828
1100000 18.204460522525714
1200000 17.7848954702754
1300000 16.845001263025036
1400000 19.145893665695326
1500000 18.77598582533478
1600000 16.999994800218687
1700000 18.603672918105715
1800000 8.614982976042707
1900000 16.157308747218355
2000000 23.02086577968162
2100000 21.42815008559972
2200000 19.905490708569367
2300000 20.33984352400253
2400000 20.66400031568293
2500000 20.26206080383844
2600000 20.126922889901355
2700000 7.995612900747895
2800000 20.761751840567918
2900000 19.871886372268683
3000000 20.602

29500000 13.939278915507925
29600000 14.881964010721076
29700000 14.728816213939536
29800000 15.932200248020768
29900000 8.856199446665189
30000000 14.806779742453607
30100000 16.795789821424385
30200000 15.090617628451358
30300000 15.499842669365515
30400000 8.62191612884768
30500000 14.454587791025883
30600000 15.596115649895124
30700000 14.480216385602048
30800000 15.49080386266975
30900000 13.396594749684006
31000000 10.169133171377547
31100000 10.044774590180639
31200000 13.351969795176
31300000 13.77515894607445
31400000 14.650350574090812
31500000 14.90747754400608
31600000 17.251696477205275
31700000 8.795771887775793
31800000 15.486599232951402
31900000 15.454292699823789
32000000 16.383386457496865
32100000 15.625115179141064
32200000 7.412851077506761
32300000 16.70928230705552
32400000 14.770464231171358
32500000 15.897661442732716
32600000 14.304243218878185
32700000 13.970561931985081
32800000 7.35871711349946
32900000 13.906467152588384
33000000 13.908832483959168
331000

59200000 12.24876094741315
59300000 12.077828927180516
59400000 5.928511599357625
59500000 11.989317302846104
59600000 11.215307223300757
59700000 11.487608541669578
59800000 7.187916967865352
59900000 12.30030617282349
60000000 12.982888218903462
60100000 13.598793127584436
60200000 13.972725819186222
60300000 13.566426594203701
60400000 13.402696432382635
60500000 13.193400761227066
60600000 12.16511102277714
60700000 12.72935678115082
60800000 5.9305234584261175
60900000 11.788271283901114
61000000 12.341364097757845
61100000 9.869406485453235
61200000 7.2633921358628575
61300000 11.483794374673527
61400000 11.922136640977836
61500000 8.87927218738231
61600000 9.36608457697392
61700000 12.134619233957732
61800000 12.531085287201273
61900000 12.268504537751102
62000000 13.298628078722246
62100000 13.512389435779513
62200000 14.02273960423212
62300000 12.664334619364205
62400000 12.850616045790016
62500000 11.675840046622184
62600000 6.564000331864264
62700000 11.64926458219586
628000

89000000 9.843166500402402
89100000 9.966637787395339
89200000 10.016658708273438
89300000 8.444985314599464
89400000 5.90718873671329
89500000 4.458731688292364
89600000 8.568456429171029
89700000 6.581102943130276
89800000 9.983159318138714
89900000 10.07419867442099
90000000 10.096757514817694
90100000 9.743173309788329
90200000 9.785950222196874
90300000 11.470482588180273
90400000 11.50594663816503
90500000 11.067431916057759
90600000 10.184794257215529
90700000 9.849823839049591
90800000 9.750121488599314
90900000 9.851965831142932
91000000 11.280712216939833
91100000 7.396957766984102
91200000 7.766398338908403
91300000 4.169441330583692
91400000 5.613498932676687
91500000 9.647238533962915
91600000 10.022853635959082
91700000 10.162258074555751
91800000 9.819939182373364
91900000 10.071467791899899
92000000 10.418790545639041
92100000 10.893605598100125
92200000 11.09168970386016
92300000 11.17221929529394
92400000 9.770408043259959
92500000 9.535314173409363
92600000 10.033904

118400000 7.611926435803123
118500000 8.094649038616568
118600000 7.889636951566504
118700000 8.847430652448468
118800000 8.879556648252645
118900000 8.024311465351671
119000000 7.959855291048013
119100000 7.764417518373371
119200000 7.9201408943411975
119300000 7.781450835450877
119400000 8.399075516767175
119500000 5.013605219476911
119600000 3.532536666342007
119700000 4.236984995530854
119800000 6.702141265585058
119900000 8.097859204233195
120000000 8.000396962335497
120100000 7.745878127417713
120200000 7.838111726627362
120300000 7.894691925165752
120400000 8.479019267925773
120500000 8.748611441081248
120600000 9.014070344545077
120700000 8.016376192045623
120800000 7.891958995507478
120900000 8.002575231323808
121000000 8.309053913510464
121100000 8.041352501758878
121200000 7.304845787463368
121300000 4.5496526139131515
121400000 3.4768001061476337
121500000 4.82507936371084
121600000 7.894578609981875
121700000 8.097235529914585
121800000 7.822014281342983
121900000 8.670941

147700000 5.15370095875142
147800000 6.414799176285541
147900000 5.590620257372111
148000000 3.035807765705499
148100000 6.484998475326612
148200000 5.955572135033816
148300000 3.037863098548184
148400000 6.194919988328168
148500000 6.402482263952434
148600000 6.530661131233613
148700000 6.859553465510166
148800000 6.69602387510401
148900000 6.485828455659395
149000000 6.480183678461058
149100000 6.622331379885704
149200000 6.133899481698459
149300000 6.043161296499775
149400000 3.5785117726746307
149500000 5.614136929237756
149600000 5.6245369979011155
149700000 4.000781551445633
149800000 4.414231148023323
149900000 6.02440149750286
150000000 6.137159417924463
150100000 2.957459223869155
150200000 5.808077382105068
150300000 6.3765969007628955
150400000 6.36848461970372
150500000 6.635632629295025
150600000 6.636723523383468
150700000 6.599170155193044
150800000 6.375610219347901
150900000 6.5704493897219
151000000 5.810615745986372
151100000 4.459240007984575
151200000 5.07167546746

176900000 2.435305391591046
177000000 4.8024647899682025
177100000 5.180104752527436
177200000 4.708063410129216
177300000 4.535009192116022
177400000 4.5920122352879975
177500000 2.5627639553638333
177600000 4.527519103497475
177700000 4.0657191201441645
177800000 4.51524168385462
177900000 3.747055247200846
178000000 3.9040094104480807
178100000 1.9068374759289093
178200000 3.8964449107811054
178300000 3.9785095524128278
178400000 4.725990654756175
178500000 4.211385001437396
178600000 4.1998354247121386
178700000 2.457159244523234
178800000 4.395323773523926
178900000 4.426212188723683
179000000 4.719993322684583
179100000 4.291371479547163
179200000 3.524203300794502
179300000 3.1915803450380564
179400000 4.181505018635958
179500000 3.9745247192499735
179600000 4.620636738454246
179700000 3.857069220682462
179800000 3.1404064391866955
179900000 2.353393906797638
180000000 3.8988497454104225
180100000 4.222990240905765
180200000 4.236594348016483
180300000 4.495823377072215
18040000

205700000 2.487817524740901
205800000 2.524950552099596
205900000 2.3684193360324173
206000000 2.350951980724156
206100000 2.2859708208325307
206200000 2.230167492279589
206300000 2.3213448659920295
206400000 1.5200816734575928
206500000 1.8495947417999374
206600000 2.3427152880159388
206700000 2.252329452403466
206800000 2.3281391707205277
206900000 2.3845828063766628
207000000 2.4872255452851157
207100000 2.361113222248789
207200000 1.9102770075183015
207300000 1.5084766324889063
207400000 1.4036367735761175
207500000 2.259981521568653
207600000 2.296588814590484
207700000 2.361510255909883
207800000 2.2571789261425734
207900000 2.1985537717351615
208000000 2.2279788251893784
208100000 2.247288631433881
208200000 0.9352028381523193
208300000 2.221621668536011
208400000 2.155882938529339
208500000 2.183178337379267
208600000 2.1945959029909172
208700000 2.3135416580524013
208800000 2.1583550997187695
208900000 2.2719883584819436
209000000 1.565339243258781
209100000 0.8655377037445009

233600000 0.12203354545772406
233700000 0.11013931913515926
233800000 0.10912885310459136
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.05862223372546832
234000000 0.020725294485946497
234100000 0.022142742200973963
234200000 0.021283436133182707
234300000 0.021969100471496587
234400000 0.021218163922333055
234500000 0.020600963378598288
234600000 0.020721494178026916
234700000 0.020027567018518846
234800000 0.019185953695158162
234900000 0.018294785851856073
235000000 0.016922323636329836
235100000 0.016809054448429082
235200000 0.013913067366957663
235300000 0.014989422988083628
235400000 0.01490122305107448
235500000 0.013226114739159743
235600000 0.01222687263550692
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.04883904276444846
235800000 0.0518480773879687
235900000 0.04711599076721403
236000000 0.04714952152611812
236100000 0.04661133697339893
236200000 0.011699506472180288
236300000 0.009778301755355464
23